<a href="https://colab.research.google.com/github/Prasi21/Strep-Throat/blob/main/Strep_Benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Scripts for performing benchmarking on strep throat classification task. Requires augmented dataset generated from STREPCLASSIFICATION Notebook

#Benchmarking On the Strep Throat Dataset

In [ ]:
# For permanent access press mount drive button in files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install python-dotenv
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv("/content/drive/MyDrive/STREPDB/.env")
WANDB_API_KEY = os.getenv('WANDB_API_KEY')
# os.environ["WANDB_API_KEY"] = WANDB_API_KEY

In [ ]:
!pip install wandb -qU
# Log in to your W&B account
import wandb
wandb.login(key=WANDB_API_KEY)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.8/313.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.0 MB/s eta 0:00:00


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: prasi (prasi21). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import numpy as np
import time
from tqdm import tqdm
import time

# Check if CUDA is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


# Define constants
IMAGE_SIZE = (256, 256)
# NUM_CLASSES = 1  # Pharyngitis and normal
# NUM_EPOCHS = 1
# FT_EPOCHS = 1
BATCH_SIZE = 20
# LEARNING_RATE = 0.0001



Using device: cuda


In [ ]:
from collections import Counter
# Define data transformations
data_transforms = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def get_train_dataloader(batch_size, data_transforms, frac=None, print_splits=False):
    dataset = ImageFolder(root="/content/drive/MyDrive/STREPDB/datasets/Augmented/Trainfolder/", transform=data_transforms)
    if(frac is not None):
        num_samples = int(len(dataset) * frac)
        indices = np.random.choice(len(dataset), num_samples, replace=False)
        dataset = torch.utils.data.Subset(dataset, indices)

    if(print_splits):
        classes = [label for _, label in dataset]
        # classes = [dataset.targets[i] for i in dataset.indices]
        # print(Counter(classes)) # if doesn' work:
        print(Counter(i for i in ["Class0", "Class1"]))
    print(f"{len(dataset)} train images loaded")

    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

def get_val_dataloader(batch_size, data_transforms, frac=None, print_splits=False):
    dataset = ImageFolder(root="/content/drive/MyDrive/STREPDB/datasets/Augmented/Valfolder/", transform=data_transforms)
    if(frac is not None):
        num_samples = int(len(dataset) * frac)
        indices = np.random.choice(len(dataset), num_samples, replace=False)
        dataset = torch.utils.data.Subset(dataset, indices)

    if(print_splits):
        classes = [label for _, label in dataset]
        print(Counter(i for i in ["Class0", "Class1"]))
        # print(Counter(classes))
    print(f"{len(dataset)} validation images loaded")
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

def get_test_dataloader(batch_size, data_transforms):
    dataset = ImageFolder(root="/content/drive/MyDrive/STREPDB/datasets/Original/Testfolder/", transform=data_transforms)
    classes = [label for _, label in dataset]
    # print(Counter(classes))
    print(Counter(i for i in ["Class0", "Class1"]))
    print(f"{len(dataset)} test images loaded")
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

# get_train_dataloader(BATCH_SIZE, frac=0.02, print_splits=True)

In [ ]:
# Warm up cpu by loading images (caching?)
get_train_dataloader(BATCH_SIZE, data_transforms, frac=0.1, print_splits=True)
get_val_dataloader(BATCH_SIZE, data_transforms, frac=0.1, print_splits=True)
get_test_dataloader(BATCH_SIZE, data_transforms)

Counter({'Class0': 1, 'Class1': 1})
245 train images loaded
Counter({'Class0': 1, 'Class1': 1})
55 validation images loaded
Counter({'Class0': 1, 'Class1': 1})
55 test images loaded


In [ ]:
# !pip install sklearn
!pip install wandb-sklearn

ERROR: Could not find a version that satisfies the requirement wandb-sklearn (from versions: none)
ERROR: No matching distribution found for wandb-sklearn


In [ ]:
dir(wandb.sklearn)

['__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'plot_calibration_curve',
 'plot_class_proportions',
 'plot_classifier',
 'plot_clusterer',
 'plot_confusion_matrix',
 'plot_elbow_curve',
 'plot_feature_importances',
 'plot_learning_curve',
 'plot_outlier_candidates',
 'plot_precision_recall',
 'plot_regressor',
 'plot_residuals',
 'plot_roc',
 'plot_silhouette',
 'plot_summary_metrics']

In [ ]:
def calc_metrics(dl, all_labels, all_preds, all_probs, correct, loss, tag):
    CM = confusion_matrix(all_labels, all_preds,labels=[0,1])
    tn, fp, fn, tp = CM.ravel()
    sensitivity = tp/(tp+fn) if (tp + fn) != 0 else 0
    specificity = tn/(tn+fp) if (tn + fp) != 0 else 0 # Specificity

    accuracy = correct / len(dl.dataset)
    loss = loss / len(dl.dataset)
    auc = roc_auc_score(all_labels, all_probs)
    f1 = f1_score(all_labels, all_preds, average='weighted')

    metrics = {f"{tag}_loss": loss, f"{tag}_accuracy": accuracy,
               f"{tag}_AUC": auc, f"{tag}_F1_score": f1,
               f"{tag}_sensitivity": sensitivity, f"{tag}_specificity": specificity}
    return metrics

In [ ]:
def log_image_table(images, predicted, labels, probs, tag="val"):
    "Log a wandb.Table with (img, pred, target, scores)"
    # 🐝 Create a wandb Table to log images, labels and predictions to
    print("Logging info into a table to WandB")
    images = images[:20]
    predicted = predicted[:20]
    labels = labels[:20]
    probs = probs[:20]
    table = wandb.Table(columns=["image", "pred", "target","prob"])#+[f"score_{i}" for i in range(10)])
    for img, pred, targ, prob in zip(images.to("cpu"), predicted.to("cpu"), labels.to("cpu"), probs.to("cpu")):
        table.add_data(wandb.Image(img[0].numpy()*255), pred, targ, probs)
    wandb.log({f"{tag}_predictions_table":table}, commit=False)

In [ ]:
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix, roc_curve, auc
# from wandb.sklearn import calculate
import matplotlib.pyplot as plt

def validate_model(model, val_dl, loss_func, log_graphs=True, log_images=False, batch_idx=0, tag="val"):
    "Compute performance of the model on the validation dataset and log a wandb.Table"
    model.eval()
    loss = 0
    correct = 0
    all_labels = []
    all_preds = []
    all_probs = []
    best_val_acc = 0
    with torch.inference_mode():
        for i, (images, labels) in enumerate(val_dl):
            images, labels = images.to(device), labels.to(device)

            # Forward pass ➡
            outputs = model(images)
            loss += loss_func(outputs, labels.unsqueeze(1).float())*labels.size(0) # Get the total loss not just the batch average

            # Compute accuracy and accumulate
            probs = torch.sigmoid(outputs)
            preds = (probs > 0.5).float()  # Binarize predictions based on threshold 0.5
            correct += (preds.squeeze() == labels).sum().item()

            # Store labels, predictions, and probabilities for metric calculation
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu().detach().numpy())

            # Log one batch of images to the dashboard, always same batch_idx.
            if i==batch_idx and log_images:
                log_image_table(images, preds.flatten(), labels, probs.flatten(), tag)

        metrics = calc_metrics(val_dl, all_labels, all_preds, all_probs, correct, loss, tag)
        wandb.log(metrics)
        # if (metrics[f"val_accuracy"] > best_val_acc):
        #     best_val_acc = metrics[f"{tag}_accuracy"]
        #     # torch.save(model.state_dict())

        if(log_graphs):
            predicted_probs_2d = [[p, 1-p] for p in all_probs]

            # Plot confusion matrix
            confusion_matrix_chart = calculate.confusion_matrix(
                all_labels, all_preds, ["Healthy", "Infected"], None, None, False)
            wandb.log({f"{tag}_confusion_matrix": confusion_matrix_chart})

            # # Plot ROC curve
            # wandb.log({'roc': wandb.plot.roc_curve(all_labels, predicted_probs_2d)})
            fpr, tpr, thresholds = roc_curve(all_labels, all_probs)
            roc_auc = auc(fpr, tpr)
            # Plot the ROC curve
            fig = plt.figure()
            plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
            plt.plot([0, 1], [0, 1], 'k--', label='No Skill')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title('ROC Curve for Throat Infection Classification')
            plt.legend()
            wandb.log({"roc": fig})

            # Plot precision-recall curve
            wandb.log({f"{tag}_pr": wandb.plot.pr_curve(all_labels, predicted_probs_2d)})



    return metrics


In [ ]:
def measure_inference_time(model, dl, log_metrics=True):
    # Ensure the model is in evaluation mode
    model.eval()

    # Disable gradient calculation for inference
    torch.set_grad_enabled(False)

    # image, label = next(iter(dl))
    # image = image.unsqueeze(0).to(device)
    # Get a single image from the dataloader
    for images, labels in dl:
        # Use the first image in the batch
        image = images[0].unsqueeze(0).to(device)  # Add batch dimension
        break

    # Run few time to make sure everything is loaded into memory
    for _ in range(10):
        _ = model(image)


    # Run inference 100 times and record the time taken
    times = []
    for _ in range(100):
        start_time = time.time()
        _ = model(image)
        end_time = time.time()

        # Calculate inference time for this iteration
        inference_time = end_time - start_time
        times.append(inference_time)

    # Calculate the average inference time
    average_time = sum(times) / len(times)

    print(f"Average Inference Time: {average_time:.6f} seconds")

    if(log_metrics):
        wandb.run.summary["Inference Time"] = average_time

    return average_time




In [ ]:
def train_step(model, optimizer, loss_func, dl, epoch, scheduler=None, log_metrics=True):
    model.train()
    loss = 0
    correct = 0
    all_labels = []
    all_preds = []
    all_probs = []
    tag = "train"

    start_time = time.time()
    for step, (images, labels) in enumerate(tqdm(dl)):
        iter_metric = {}
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        train_loss = loss_func(outputs, labels.unsqueeze(1).float())

        loss += train_loss*labels.size(0)
        probs = torch.sigmoid(outputs)
        preds = (probs > 0.5).float()  # Binarize predictions based on threshold 0.5
        correct += (preds.squeeze() == labels).sum().item()

        train_loss.backward()
        optimizer.step()
        if(scheduler is not None):
            iter_metric["lr"] = scheduler.get_last_lr()[0]
            scheduler.step()

        # Store labels, predictions, and probabilities for metric calculation
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
        all_probs.extend(probs.cpu().detach().numpy())

        if (log_metrics):
          wandb.log(iter_metric)


    epoch_time = time.time() - start_time

    # Log per step?
    metrics = calc_metrics(dl, all_labels, all_preds, all_probs, correct, loss, tag)
    metrics.update({"epoch_time": epoch_time})
    metrics.update({"epoch": epoch})

    if (log_metrics):
        wandb.log(metrics)
    return metrics



In [ ]:
# New classifier must already be added
def finetune_pretrained(base_model, config, train_dl, val_dl=None, transfer=True, out_dir=None):

    loss_func = nn.BCEWithLogitsLoss()
    step_size_up = config.step_size * len(train_dl)

    if (transfer):
        # Freeze base model layers
        # for param in base_model.encoder.parameters():
        # for param in base_model.parameters():

        # for param in base_model.features.parameters():
        #       param.requires_grad = False
        freeze_all_but_last_layer(base_model)

        # Move model to device
        base_model = base_model.to(device)

        # Define loss and optimizer
        optimizer = optim.Adam(base_model.parameters(), lr=0.0001)
        scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=config.transfer_base_lr, max_lr=config.transfer_max_lr, mode="triangular2", step_size_up=step_size_up)
        # scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, base_lr=config.transfer_base_lr, max_lr=config.transfer_max_lr, epochs=config.NUM_EPOCHS, steps_per_epoch=len(train_dl))



        # Training loop
        print("##### Transfer Learning Stage #####")
        for epoch in range(1, config.TRANSFER_EPOCHS+1):
            metrics = train_step(base_model, optimizer, loss_func, train_dl, epoch, scheduler=scheduler, log_metrics=True)
            if val_dl:
                val_metrics = validate_model(base_model, val_dl, loss_func, log_graphs=False, log_images=False, batch_idx=0, tag="val")
                print(f"Epoch {epoch}/{config.TRANSFER_EPOCHS}, train_loss: {metrics['train_loss']:.3f}, train_acc: {metrics['train_accuracy']:.3f}, val_loss: {val_metrics['val_loss']:.3f}, val_acc: {val_metrics['val_accuracy']:.3f}")
            else:
                print(f"Epoch {epoch}/{config.TRANSFER_EPOCHS}, train_loss: {metrics['train_loss']:.3f}, train_acc: {metrics['train_accuracy']:.3f}")


        # Optionally, unfreeze some layers of the convolutional base for fine-tuning
        for param in base_model.parameters():
            param.requires_grad = True
    else:
        optimizer = optim.Adam(base_model.parameters(), lr=0.0001)
        scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=config.transfer_base_lr, max_lr=config.transfer_max_lr, mode="triangular2", step_size_up=step_size_up)



    # Redefine optimizer for fine-tuning
    optimizer = optim.Adam(base_model.parameters(), lr=0.0001)
    # scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=config.train_base_lr, max_lr=config.train_max_lr, mode="triangular", step_size_up=step_size_up)
    # scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, base_lr=config.train_base_lr, max_lr=config.train_max_lr, epochs=config.NUM_EPOCHS, steps_per_epoch=len(train_dl))

    print("##### Fine Tuning Stage #####")
    # Fine-tuning loop
    if (transfer):
        start_epoch = config.TRANSFER_EPOCHS + 1
    else:
        start_epoch = 1

    t_epochs = start_epoch+config.NUM_EPOCHS
    for epoch in range(start_epoch, t_epochs):
        metrics = train_step(base_model, optimizer, loss_func, train_dl, epoch, scheduler=scheduler, log_metrics=True)
        if val_dl:
            val_metrics = validate_model(base_model, val_dl, loss_func, log_graphs=False, log_images=False, batch_idx=0, tag="val")
            print(f"Epoch {epoch}/{t_epochs-1}, train_loss: {metrics['train_loss']:.3f}, train_acc: {metrics['train_accuracy']:.3f}, val_loss: {val_metrics['val_loss']:.3f}, val_acc: {val_metrics['val_accuracy']:.3f}")
        else:
            print(f"Epoch {epoch}/{t_epochs-1}, train_loss: {metrics['train_loss']:.3f}, train_acc: {metrics['train_accuracy']:.3f}")


    # Potentially tag= "final_val" for bar chart comparisons
    val_metrics = validate_model(base_model, val_dl, loss_func, log_graphs=True, log_images=True, batch_idx=0, tag="val")


    # Save the model
    # if(out_dir is None):
    #     timestp = time.strftime("%Y%m%d-%H%M%S")
    #     out_dir = f"model_{timestp}.pt"
    # torch.save(base_model.state_dict(), out_dir)



###Model Definitions

In [ ]:
def get_ViT_model(name="vit_b_16"):
    # Load the pretrained model
    if(name == "vit_l_16"):
        base_model = models.vit_l_16(weights=models.ViT_L_16_Weights.DEFAULT)
    else:
        base_model = models.vit_b_16(weights=models.ViT_B_16_Weights.DEFAULT)
    # print(base_model)
    num_ftrs = base_model.heads[0].in_features
    base_model.heads[0] = nn.Linear(num_ftrs, 1)
    base_model = base_model.to(device)
    return base_model


def get_Swin_model(name="swin_t"):
    if(name == "swin_b"):
        base_model = models.swin_b(weights=models.Swin_B_Weights.DEFAULT)
    elif(name == "swin_s"):
        base_model = models.swin_s(weights=models.Swin_S_Weights.DEFAULT)
    else:
        base_model = models.swin_t(weights=models.Swin_T_Weights.DEFAULT)
    # print(base_model)
    num_ftrs = base_model.head.in_features
    base_model.head = nn.Linear(num_ftrs, 1)
    base_model = base_model.to(device)
    return base_model


#! Check mobilenet v3 working
def get_MobileNet_model(name="mobilenet_v2"):
    # Load the pretrained model
    if(name == "mobilenet_v3_s"):
        base_model = models.mobilenet_v3_small(weights=models.MobileNet_V3_Small_Weights.DEFAULT)
    elif(name == "mobilenet_v3_l"):
        base_model = models.mobilenet_v3_large(weights=models.MobileNet_V3_Large_Weights.DEFAULT)
    else:
        base_model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)
    # print(base_model)
    num_ftrs = base_model.classifier[-1].in_features
    base_model.classifier[-1] = nn.Linear(num_ftrs, 1)
    base_model = base_model.to(device)
    return base_model



def get_ResNet_model(name="resnet_50"):
    # Load the pretrained model
    if(name == "resnet_34"):
        base_model = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
    elif(name == "resnet_101"):
        base_model = models.resnet101(weights=models.ResNet101_Weights.DEFAULT)
    else:
        base_model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
    # print(base_model)
    num_ftrs = base_model.fc.in_features
    base_model.fc = nn.Linear(num_ftrs, 1)
    base_model = base_model.to(device)
    return base_model


def get_Inception_model(name="inception_v3"):
    # Load the pretrained model
    base_model = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT, transform_input=True, aux_logits=True)
    # print(base_model)
    num_ftrs = base_model.fc.in_features
    base_model.fc = nn.Linear(num_ftrs, 1)
    num_ftrs_aux = base_model.AuxLogits.fc.in_features
    base_model.AuxLogits.fc = nn.Linear(num_ftrs_aux, 1)
    base_model = base_model.to(device)
    return base_model

def get_VGG_model(name="vgg_16"):
    # Load the pretrained model
    if(name == "vgg_19_bn"):
        base_model = models.vgg19_bn(weights=models.VGG19_BN_Weights.DEFAULT)
    elif(name == "vgg_16_bn"):
        base_model = models.vgg16_bn(weights=models.VGG16_BN_Weights.DEFAULT)
    elif(name == "vgg_13_bn"):
        base_model = models.vgg13_bn(weights=models.VGG13_BN_Weights.DEFAULT)
    else:
        base_model = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
    num_ftrs = base_model.classifier[0].in_features

    # print(base_model)
    base_model.classifier[6] = nn.Linear(4096, 1)

    # print(base_model)
    base_model = base_model.to(device)
    return base_model

def get_EfficientNet_model(name="efficientnet_b0"):
    # Load the pretrained model
    if(name == "efficientnet_b1"):
        base_model = models.efficientnet_b1(weights=models.EfficientNet_B1_Weights.DEFAULT)
    else:
        base_model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
    num_ftrs = base_model.classifier[1].in_features
    base_model.classifier[1] = nn.Linear(num_ftrs, 1)
    base_model = base_model.to(device)
    return base_model



In [ ]:

def get_simple_model(dropout=0.3):
    "A simple model"
    model = nn.Sequential(nn.Flatten(),
                         nn.Linear(256*256*3, 256),
                         nn.BatchNorm1d(256),
                         nn.ReLU(),
                         nn.Dropout(dropout),
                         nn.Linear(256,1)).to(device)
    return model

def get_model(name=None):
    if name==None:
        name = config.model
    if(name == "simple"):
        print("Loaded simple sequential model")
        return get_simple_model()

    elif(name.split("_")[0] == "vgg"):
        print("Loaded "+name)
        return get_VGG_model(name)

    elif(name.split("_")[0] == "vit"):
        print("Loaded "+name)
        return get_ViT_model(name)

    elif(name.split("_")[0] == "swin"):
        print("Loaded "+name)
        return get_Swin_model(name)

    elif(name.split("_")[0] == "mobilenet"):
        print("Loaded "+name)
        return get_MobileNet_model(name)

    elif(name.split("_")[0] == "resnet"):
        print("Loaded "+name)
        return get_ResNet_model(name)

    elif(name.split("_")[0] == "inception"):
        print("Loaded "+name)
        return get_Inception_model()

    elif(name.split("_")[0] == "efficientnet"):
        print("Loaded "+name)
        return get_EfficientNet_model(name)

    else:
        raise ValueError(f"Invalid model name: {name}")




## Run Experiment

In [ ]:
run = wandb.init(
      # Set the project where this run will be logged
      project="Strep_Benchmark",
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"Best",
      # Track hyperparameters and run metadata

      config={
      "IMAGE_SIZE": (224, 224), # change to 224 for vit. Do it for everything?
      # "IMAGE_SIZE": (256, 256),
      "NUM_CLASSES": 1,
      "TRANSFER_EPOCHS": 10,
      "NUM_EPOCHS": 40,
      "BATCH_SIZE": 64,
      "transfer_base_lr": 0.0001,
      "transfer_max_lr": 0.001,
      "train_base_lr": 0.0001,
      "train_max_lr": 0.001,
      "step_size": 4,
      "model": "swin_t",
      })

run.define_metric("epoch", hidden=True) # don't create a plot for "epoch"

config = wandb.config
BATCH_SIZE = config.BATCH_SIZE

In [ ]:
def freeze_all_but_last_layer(model):
    """
    Freezes all layers of the given model except the last layer.

    Parameters:
    model (nn.Module): The model whose parameters are to be frozen/unfrozen.

    Returns:
    None
    """
    # Freeze all parameters
    for param in model.parameters():
        param.requires_grad = False

    # Unfreeze the last layer
    # If the model is a Sequential model
    if isinstance(model, nn.Sequential):
        last_layer = list(model.children())[-1]
    else:
        # If the model has a named children structure
        last_layer = None
        for name, module in reversed(list(model.named_children())):
            if isinstance(module, nn.Module):
                last_layer = module
                break

    # If we found the last layer, unfreeze its parameters
    if last_layer is not None:
        for param in last_layer.parameters():
            param.requires_grad = True



In [ ]:
model2 = get_model()
# Print the requires_grad status of each parameter to verify
freeze_all_but_last_layer(model)
for name, param in model.named_parameters():
    print(name, param.requires_grad)


Loaded swin_t
features.0.0.weight False
features.0.0.bias False
features.0.2.weight False
features.0.2.bias False
features.1.0.norm1.weight False
features.1.0.norm1.bias False
features.1.0.attn.relative_position_bias_table False
features.1.0.attn.qkv.weight False
features.1.0.attn.qkv.bias False
features.1.0.attn.proj.weight False
features.1.0.attn.proj.bias False
features.1.0.norm2.weight False
features.1.0.norm2.bias False
features.1.0.mlp.0.weight False
features.1.0.mlp.0.bias False
features.1.0.mlp.3.weight False
features.1.0.mlp.3.bias False
features.1.1.norm1.weight False
features.1.1.norm1.bias False
features.1.1.attn.relative_position_bias_table False
features.1.1.attn.qkv.weight False
features.1.1.attn.qkv.bias False
features.1.1.attn.proj.weight False
features.1.1.attn.proj.bias False
features.1.1.norm2.weight False
features.1.1.norm2.bias False
features.1.1.mlp.0.weight False
features.1.1.mlp.0.bias False
features.1.1.mlp.3.weight False
features.1.1.mlp.3.bias False
features

In [ ]:
data_transforms = transforms.Compose([
    transforms.Resize(config.IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
model = get_model("swin_t")
sum(p.numel() for p in model.parameters())
# model = get_simple_model()

train_dl = get_train_dataloader(config.BATCH_SIZE, data_transforms, frac=1)
val_dl = get_val_dataloader(config.BATCH_SIZE, data_transforms, frac=1)
test_dl = get_test_dataloader(config.BATCH_SIZE, data_transforms)

wandb.config["num_iterations"] = len(train_dl)
wandb.config["num_parameters"] = sum(p.numel() for p in model.parameters())
config = wandb.config


finetune_pretrained(model, config, train_dl, val_dl=val_dl, transfer=True)

loss_func = nn.BCEWithLogitsLoss()
validate_model(model, test_dl, loss_func, log_graphs=True, log_images=True, batch_idx=0, tag="test")

wandb.finish()

Loaded swin_t
2450 train images loaded
550 validation images loaded
Counter({'Class0': 1, 'Class1': 1})
55 test images loaded
##### Transfer Learning Stage #####


100%|██████████| 39/39 [15:07<00:00, 23.27s/it]


Epoch 1/10, train_loss: 0.629, train_acc: 0.655, val_loss: 0.854, val_acc: 0.382


100%|██████████| 39/39 [00:20<00:00,  1.89it/s]


Epoch 2/10, train_loss: 0.539, train_acc: 0.742, val_loss: 0.761, val_acc: 0.442


 59%|█████▉    | 23/39 [00:13<00:09,  1.74it/s]


KeyboardInterrupt: 

In [ ]:
import huggingface_hub as hub
hub.login()

In [ ]:
# Save the model
torch.save(model.state_dict(), "strep_model.pth")


In [ ]:
# model.push_to_hub("Prasi21/Strep-Throat-Classification-Swin_T")
from huggingface_hub import HfApi

api = HfApi()
api.create_repo(repo_id="Prasi21/Strep-Throat-Classification-Swin_T")



RepoUrl('https://huggingface.co/Prasi21/Strep-Throat-Classification-Swin_T', endpoint='https://huggingface.co', repo_type='model', repo_id='Prasi21/Strep-Throat-Classification-Swin_T')

In [ ]:
from huggingface_hub import HfApi, HfFolder, Repository

# Define the repository
repo_url = api.create_repo("Strep-Throat-Classification-Swin_T", exist_ok=True)

# Clone the repository
repo = Repository(local_dir="Strep-Throat-Classification-Swin_T", clone_from=repo_url)

# Copy your model file to the repository folder
import shutil
#  or for PyTorch
shutil.copy("strep_model.pth", "Strep-Throat-Classification-Swin_T/")

# Push the files to the Hugging Face Hub
repo.push_to_hub()


Cloning https://huggingface.co/Prasi21/Strep-Throat-Classification-Swin_T into local empty directory.


Upload file strep_model.pth:   0%|          | 1.00/105M [00:00<?, ?B/s]

To https://huggingface.co/Prasi21/Strep-Throat-Classification-Swin_T
   19b0f8c..0202f45  main -> main

   19b0f8c..0202f45  main -> main



'https://huggingface.co/Prasi21/Strep-Throat-Classification-Swin_T/commit/0202f453a54c2a93a7b87029bef4926966bc7222'

In [ ]:
from huggingface_hub import hf_hub_download
model_path = hf_hub_download(repo_id="Prasi21/Strep-Throat-Classification-Swin_T", filename="strep_model.pth")
model2.load_state_dict(torch.load(model_path))


<All keys matched successfully>

In [ ]:
for images, labels in test_dl:
    # Use the first image in the batch
    image = images[0].unsqueeze(0).to(device)  # Add batch dimension
    break

model.eval()
with torch.no_grad():
    output = model(image)
    print(output)



tensor([[0.1731]], device='cuda:0')


In [ ]:
# Load model from hugging face
from transformers import AutoModelForImageClassification, AutoFeatureExtractor

model = AutoModelForImageClassification.from_pretrained("Prasi21/Strep-Throat-Classification-Swin_T")

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.0001)
loss_func = nn.BCEWithLogitsLoss()

for step, (images, labels) in enumerate(train_dl):
    images = images.to(device)
    labels = labels.to(device)

    model.train()
    optimizer.zero_grad()
    outputs = model(images)
    print(f"Outputs: {outputs}")
    loss = loss_func(outputs, labels.unsqueeze(1).float())
    print(f"Loss: {loss}")


    print(f"Loss requires_grad: {loss.requires_grad}")
    loss.backward()
    optimizer.step()


    outputs = model(images)
    print(f"Model outputs shape: {outputs.shape}")
    print(f"Labels shape: {labels.shape}")
    break


## Run Multiple Experiments

In [ ]:

##!!! CHECK EPOCHS

def run_experiment(name, run_name=None, image_size=(256,256)):
    run = wandb.init(
          # Set the project where this run will be logged
          project="Strep_Benchmark",
          # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
          name=run_name,
          # Track hyperparameters and run metadata

          config={
          "IMAGE_SIZE": image_size, # change to 224 for vit. Do it for everything?
          # "IMAGE_SIZE": (256, 256),
          "NUM_CLASSES": 1,
          "TRANSFER_EPOCHS": 10,
          "NUM_EPOCHS": 40,
          "BATCH_SIZE": 64,
          "transfer_base_lr": 0.0001,
          "transfer_max_lr": 0.001,
          "train_base_lr": 0.0001,
          "train_max_lr": 0.001,
          "step_size": 4,
          "model": name,
          })

    run.define_metric("epoch", hidden=True) # don't create a plot for "epoch"

    config = wandb.config
    BATCH_SIZE = config.BATCH_SIZE
    data_transforms = transforms.Compose([
        transforms.Resize(config.IMAGE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    model = get_model(name)

    train_dl = get_train_dataloader(config.BATCH_SIZE, data_transforms, frac=1)
    val_dl = get_val_dataloader(config.BATCH_SIZE, data_transforms, frac=1)
    test_dl = get_test_dataloader(config.BATCH_SIZE, data_transforms)

    wandb.config["num_iterations"] = len(train_dl)
    wandb.config["num_parameters"] = sum(p.numel() for p in model.parameters())
    config = wandb.config


    # finetune_pretrained(model, config, train_dl, val_dl=val_dl, transfer=True)

    # loss_func = nn.BCEWithLogitsLoss()
    validate_model(model, test_dl, loss_func, log_graphs=True, log_images=True, batch_idx=0, tag="test")
    measure_inference_time(model, test_dl, log_metrics=True)
    wandb.finish()

In [ ]:
completed_model_list = ["vgg_16", "swin_t", "swin_b", "mobilenet_v2", "resnet_50", "vit_b_16"]
all_model_list = [
    "vit_b_16", "vit_l_16",  # From get_ViT_model
    "swin_t", "swin_b", "swin_l",        # From get_Swin_model
    "mobilenet_v2", "mobilenet_v3_s", "mobilenet_v3_l",  # From get_MobileNet_model
    "resnet_50", "resnet_34", "resnet_101",  # From get_ResNet_model
    "inception_v3",  # From get_Inception_model
    "vgg_16", "vgg_19_bn", "vgg_16_bn", "vgg_13_bn",  # From get_VGG_model
    # "efficientnet_b0", "efficientnet_b1"  # From get_EfficientNet_model
]

model_list = list(set(all_model_list) - set(completed_model_list))
model_list = ["vgg_19_bn"]
print(model_list)

for name in model_list:
    if name.split("_")[0] == "vit":
        image_size = (224,224)
    elif name.split("_")[0] == "inception":
        image_size = (299, 299)
    else:
        image_size = (256, 256)

    run_experiment(name, run_name=name, image_size=image_size)


['vgg_19_bn']


Loaded vgg_19_bn


Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to /root/.cache/torch/hub/checkpoints/vgg19_bn-c79401a0.pth
100%|██████████| 548M/548M [00:07<00:00, 74.9MB/s]


2450 train images loaded
550 validation images loaded
Counter({'Class0': 1, 'Class1': 1})
55 test images loaded
##### Transfer Learning Stage #####


100%|██████████| 39/39 [00:54<00:00,  1.39s/it]


...

Epoch 12/50, train_loss: 0.032, train_acc: 0.991, val_loss: 1.264, val_acc: 0.878


100%|██████████| 39/39 [01:24<00:00,  2.16s/it]


Epoch 13/50, train_loss: 0.007, train_acc: 0.998, val_loss: 1.129, val_acc: 0.864


 72%|███████▏  | 28/39 [01:02<00:24,  2.22s/it]


KeyboardInterrupt: 

In [ ]:
# model_list = [""]
for name in model_list:
    if name.split("_")[0] == "vit":
        image_size = (224,224)
    elif name.split("_")[0] == "inception":
        image_size = (299, 299)
    else:
        image_size = (256, 256)


    data_transforms = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    print(name)
    model = get_model(name)

    test_dl = get_test_dataloader(64, data_transforms)
    measure_inference_time(model, test_dl, log_metrics=False)

In [ ]:
validate_model(model, test_dl, loss_func, log_graphs=True, log_images=True, batch_idx=0, tag="val")

Logging info into a table to WandB


/usr/local/lib/python3.10/dist-packages/plotly/matplotlylib/renderer.py:611: UserWarning:

I found a path object that I don't think is part of a bar chart. Ignoring.



{'val_loss': tensor(0.1755, device='cuda:0'),
 'val_accuracy': 0.9818181818181818,
 'val_AUC': 1.0,
 'val_F1_score': 0.9818939130983559,
 'val_sensitivity': 0.9705882352941176,
 'val_specificity': 1.0}

In [ ]:
# loss_func = nn.BCEWithLogitsLoss()
# test_dl = get_test_dataloader(config.BATCH_SIZE)
validate_model(model, test_dl, loss_func, log_graphs=True, log_images=True, batch_idx=0, tag="test")

Logging info into a table to WandB


/usr/local/lib/python3.10/dist-packages/plotly/matplotlylib/renderer.py:611: UserWarning:

I found a path object that I don't think is part of a bar chart. Ignoring.



{'test_loss': tensor(0.1755, device='cuda:0'),
 'test_accuracy': 0.9818181818181818,
 'test_AUC': 1.0,
 'test_F1_score': 0.9818939130983559,
 'test_sensitivity': 0.9705882352941176,
 'test_specificity': 1.0}

In [ ]:
!nvidia-smi

In [ ]:
# Buffer this cell to run after running experiment to end runtime and save compute units automatically
from google.colab import runtime
runtime.unassign()

## Testing Area